In [2]:
import cv2
import numpy as np

In [3]:
SHOW_TIMER = 1000

In [4]:
frame = cv2.imread("../frames/masked_frame.png")
hsv_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2HSV)

In [5]:
cv2.imshow("frame", frame)
cv2.waitKey(SHOW_TIMER)
cv2.destroyAllWindows()

In [6]:
cv2.imshow("Vue", hsv_frame[:,:,2])
cv2.waitKey(0)
cv2.destroyAllWindows() 

In [7]:
cv2.imshow("Saturation", hsv_frame[:,:,1])
cv2.waitKey(0)
cv2.destroyAllWindows() 

In [8]:
cv2.imshow("Hue", hsv_frame[:,:,0])
cv2.waitKey(0)
cv2.destroyAllWindows() 

In [9]:
COLORS = ["blue", "red", "pink", "brown"]

In [10]:
MAX_BLUE = (25, 255, 255)
MIN_BLUE = (10, 0, 30)
mask_blue = cv2.inRange(hsv_frame, MIN_BLUE, MAX_BLUE) # 

In [11]:
cv2.imshow("mask blue", mask_blue) 
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
MAX_RED = (130, 255, 200)
MIN_RED = (110, 0, 0)
mask_red = cv2.inRange(hsv_frame, MIN_RED, MAX_RED) #

In [13]:
cv2.imshow("mask red", mask_red) 
cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
MAX_PINK = (130, 255, 255)
MIN_PINK = (110, 0, 200)
mask_pink = cv2.inRange(hsv_frame, MIN_PINK, MAX_PINK) # TODO SELECT LARGEST PATCH

In [15]:
cv2.imshow("mask pink", mask_pink) 
cv2.waitKey(0)
cv2.destroyAllWindows()

In [16]:
MAX_BROWN = (110, 255, 150)
MIN_BROWN = (90, 0, 0)
mask_brown = cv2.inRange(hsv_frame, MIN_BROWN, MAX_BROWN) # TODO SELECT LARGEST PATCH OR OTHER FIX

In [17]:
cv2.imshow("mask brown", mask_brown) 
cv2.waitKey(0)
cv2.destroyAllWindows()

In [18]:
MAX_YELLOW = (100, 255, 255)
MIN_YELLOW = (90, 0, 150)
mask_yellow = cv2.inRange(hsv_frame, MIN_YELLOW, MAX_YELLOW) # TODO SELECT LARGEST PATCH

In [19]:
cv2.imshow("mask yellow", mask_yellow) 
cv2.waitKey(0)
cv2.destroyAllWindows()

In [20]:
MAX_GREEN = (50, 255, 255)
MIN_GREEN = (30, 0, 0)
mask_green = cv2.inRange(hsv_frame, MIN_GREEN, MAX_GREEN) # TODO SELECT LARGEST PATCH OR OTHER FIX

In [21]:
cv2.imshow("mask brown", mask_green) 
cv2.waitKey(0)
cv2.destroyAllWindows()

In [22]:
MAX_WHITE = (85, 255, 255)
MIN_WHITE = (70, 0, 100)
mask_white = cv2.inRange(hsv_frame, MIN_WHITE, MAX_WHITE) # TODO SELECT LARGEST PATCH OR OTHER FIX

In [23]:
cv2.imshow("mask white", mask_white) 
cv2.waitKey(0)
cv2.destroyAllWindows()

In [24]:
MAX_BLACK = (50, 255, 50)
MIN_BLACK = (1, 0, 0)
mask_black = cv2.inRange(hsv_frame, MIN_BLACK, MAX_BLACK) # TODO SELECT LARGEST PATCH OR OTHER FIX

In [25]:
cv2.imshow("mask black", mask_black) 
cv2.waitKey(0)
cv2.destroyAllWindows()

In [26]:
def matches_window(frame, color_min_lower, color_max_lower, color_min_upper, color_max_upper):
    lower = cv2.inRange(frame, color_min_lower, color_max_lower)
    upper = cv2.inRange(frame, color_min_upper, color_max_upper)

    cv2.imshow("lower", lower) 
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    lower_matched_x, lower_matched_y = np.where(lower==255)

    matchings = []
    for lower_candidate_x, lower_candidate_y in zip(lower_matched_x, lower_matched_y):
        if lower[lower_candidate_x, lower_candidate_y] > 0:
            if upper[lower_candidate_x-1, lower_candidate_y] == 255:
                matchings.append((lower_candidate_x-1, lower_candidate_y))
        else: print("image window out of bounds")
    return matchings
            

In [27]:
def visualize_points(image, points, color):
    # Takes a greyscale image and paints some points in color on top
    image = image.copy()
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    for x,y in points:
        image[x,y] = color
    return image

In [28]:
MIN_RED, MAX_RED

((110, 0, 0), (130, 255, 200))

In [29]:
MAX_RED_HIGHLIGHT = (130, 255, 255)
MIN_RED_HIGHLIGHT = (110, 0, 180)
matchings = matches_window(hsv_frame, MIN_RED, MAX_RED, MIN_RED_HIGHLIGHT, MAX_RED_HIGHLIGHT)
matchings

[(419, 632),
 (419, 647),
 (420, 632),
 (420, 646),
 (420, 647),
 (421, 633),
 (421, 643),
 (421, 644),
 (421, 645),
 (421, 646),
 (422, 633),
 (422, 634),
 (422, 635),
 (422, 636),
 (422, 637),
 (422, 638),
 (422, 640),
 (422, 641),
 (422, 642),
 (422, 643),
 (422, 644),
 (423, 634),
 (423, 635),
 (423, 641),
 (423, 643),
 (423, 644),
 (424, 633),
 (424, 634),
 (424, 641),
 (424, 643),
 (425, 633),
 (425, 634),
 (425, 641),
 (426, 633),
 (426, 642),
 (426, 644),
 (427, 633),
 (427, 635),
 (427, 640),
 (427, 642),
 (427, 643),
 (427, 644),
 (428, 632),
 (428, 633),
 (428, 635),
 (428, 640),
 (428, 642),
 (428, 643),
 (428, 644),
 (429, 632),
 (429, 633),
 (429, 635),
 (429, 636),
 (429, 637),
 (429, 638),
 (429, 639),
 (429, 642),
 (429, 643),
 (429, 644),
 (430, 636),
 (432, 666),
 (433, 655),
 (433, 656),
 (433, 657),
 (433, 665),
 (433, 666),
 (434, 655),
 (434, 656),
 (434, 657),
 (434, 662),
 (434, 664),
 (434, 665),
 (434, 666),
 (435, 655),
 (435, 656),
 (435, 657),
 (435, 658),

In [30]:
red_highlights = visualize_points(mask_red, matchings, (0,0,255))

In [31]:
cv2.imshow("matchings", red_highlights) 
cv2.waitKey(0)
cv2.destroyAllWindows()

In [33]:
# for all ball colors (if red x 15) do
# move the window over candiate pixels or whole frame
# get hte mean of the highlight pixels
# return [(color, highlight_coord, image_mask), ()]

In [34]:
hsv_frame.shape

(720, 1280, 3)

In [35]:
import numpy as np

In [36]:
cv2.inRange(np.array([[[2,2,20]]]), (0,0,0), (10,10,10))

array([[0]], dtype=uint8)

In [37]:
def get_highlight(frame, ):
    pass

In [38]:
np.where(np.array([1,2,3, 2])==2)

(array([1, 3], dtype=int64),)